In [27]:
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.metrics import average_precision_score
import multiprocessing
from tqdm.auto import tqdm
import scipy.sparse as sp
from rdkit import Chem
import numpy as np
from rdkit.Chem import rdFingerprintGenerator
from wandb.integration.xgboost import WandbCallback
import os 
import re
import pyarrow.parquet as pq
from scipy.sparse import coo_matrix, csr_matrix, hstack, vstack, save_npz, load_npz
import wandb

In [28]:
parquet_file = pq.ParquetFile('../../test.parquet')
ecfp_gen = rdFingerprintGenerator.GetMorganGenerator(radius=3, fpSize=2048)

In [46]:
for df in parquet_file.iter_batches():
    df = df.to_pandas()
    mol = [Chem.MolFromSmiles(x.replace("Dy","C")) for x in df['molecule_smiles']]
    ecfp = [ecfp_gen.GetFingerprint(x).ToList() for x in mol]
    result = csr_matrix(ecfp)
    one_hot = np.zeros((len(df), 3))
    one_hot[df['protein_name']=="BRD4",0] = 1
    one_hot[df['protein_name']=="HSA",1] = 1
    one_hot[df['protein_name']=="sEH",2] = 1
    result = hstack([result, one_hot])
    break
    

In [49]:
dtrain = xgb.DMatrix(result)


In [48]:
xgb.model.lo

In [47]:
result

<65536x2051 sparse matrix of type '<class 'numpy.float64'>'
	with 6326773 stored elements in COOrdinate format>

In [34]:
one_hot.shape

(3,)

In [35]:
result.shape

(1, 2048)